## This notebook only seeks to draw the workflow out

In [1]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.llms import LLM
from llama_index.core.workflow import (
    Workflow,
    Context,
    Event,
    StartEvent,
    StopEvent,
    step
)

In [2]:
from llama_index.tools.tavily_research import TavilyToolSpec
from llama_index.llms.openai import OpenAI

In [3]:
from llama_index.utils.workflow import (
    draw_all_possible_flows
)

In [5]:
from typing import Optional, Annotated, List, Any

class SearchEvent(Event):
    """Requires the LLM to do an online search to answer the question"""
    query: Annotated[str, "The user's query"]

class AnswerEvent(Event):
    """Allows the LLM to answer the question without searching"""
    query: Annotated[str, "The user's query"]

class ResponseEvent(Event):
    """Collects LLM response"""
    query: Annotated[str, "The user's query"]
    answer: Annotated[str, "The LLM's response"]

In [6]:
import os

llm = OpenAI(model="gpt-4o-mini")

### Define tools
search_tool_spec = TavilyToolSpec(api_key=os.getenv("TAVILY"))
search_tools = search_tool_spec.to_tool_list()

### Define events
class SearchEvent(Event):
    """Requires the LLM to do an online search to answer the question"""
    query: Annotated[str, "The user's query"]

class AnswerEvent(Event):
    """Allows the LLM to answer the question without searching"""
    query: Annotated[str, "The user's query"]

class ResponseEvent(Event):
    """Collects LLM response"""
    query: Annotated[str, "The user's query"]
    answer: Annotated[str, "The LLM's response"]

/opt/anaconda3/envs/llamaindex/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [7]:
class MixtureOfAnswers(Workflow):
    def __init__(
        self, 
        *args: Any,
        llm: Optional[LLM] = llm,
        **kwargs: Any
    ):
        """Class constructor. Takes in an llm instance and constructs 
        1. A function calling agent with search tools
        2. A simple chat engine instance
        3. A common memory instance across the workflow

        Args:
            llm (Optional[LLM], optional): LLM instance. Defaults to Settings.llm.
        """
        super().__init__(*args, **kwargs)
        self.llm = llm
        self.search_agent_worker = FunctionCallingAgentWorker.from_tools(
            tools = search_tools,
            llm = self.llm
        )
        self.search_agent = self.search_agent_worker.as_agent()
        self.answer_without_search_engine = SimpleChatEngine.from_defaults(
            llm = self.llm
        )
        self.history: List[ChatMessage] = []
    
    @step()
    async def route_to_llm(
        self,
        ev: StartEvent
    ) -> SearchEvent | AnswerEvent:
        """Generates a search event and an answer event once given a start event"""
        
        ## Update memory
        self.history.append(
            ChatMessage(
                role = MessageRole.USER,
                content = ev.query
            )
        )
        
        ## Routes to both events. But you can also write a router component to decide 
        ## which event to route to.
        self.send_event(SearchEvent(query = ev.query))
        self.send_event(AnswerEvent(query = ev.query))
    
    @step()
    async def search_and_answer(
        self,
        ev: SearchEvent
    ) -> ResponseEvent:
        """Uses the tavily search tool to answer the question"""
        
        ## Synthesize response
        response = await self.search_agent.achat(
            ev.query, 
            chat_history = self.history
        )
        
        ## [OPTIONAL] Show intermediate response in the frontend
        # await cl.Message(content="ANSWER WITH SEARCH: " + str(response)).send()
        
        ## Update memory
        self.history.append(
            ChatMessage(
                role = MessageRole.ASSISTANT,
                content = "ANSWER WITH SEARCH: " + str(response)
            )
        )
        
        return ResponseEvent(query = ev.query, answer = str(response))

    @step()
    async def simply_answer(
        self,
        ev: AnswerEvent
    ) -> ResponseEvent:
        """Uses the LLM to simple answer the question"""
        
        ## Synthesize response
        response = await self.answer_without_search_engine.achat(
            ev.query, 
            chat_history = self.history
        )
        
        ## [OPTIONAL] Show intermediate response in the frontend
        # await cl.Message(content="ANSWER WITHOUT SEARCH: " + str(response)).send()
        
        ## Update memory
        self.history.append(
            ChatMessage(
                role = MessageRole.ASSISTANT,
                content = "ANSWER WITHOUT SEARCH: " + str(response)
            )
        )
        
        return ResponseEvent(query = ev.query, answer = str(response))
    
    @step()
    async def compile(
        self,
        ctx: Context,
        ev: ResponseEvent
    ) -> StopEvent:
        """Compiles and summarizes answers from all response events"""
        
        ## There are 2 response events from routing to 2 different agents. This can
        ## also be a dynamic number of events.
        ready = ctx.collect_events(ev, [ResponseEvent] * 2) 
        
        if ready is None:
            return None
        
        response = await self.llm.acomplete(
            f"""
            A user has asked us a question and we have responded accordingly using a 
            search tool and without using a search tool. Your job is to decide which 
            response best answered the question and summarize the response into a crisp 
            reply. If both responses answered the question, summarize both responses
            into a single answer.
            
            The user's query was: {ev.query}
            
            The responses are:
            {ready[0].answer} &
            {ready[1].answer}
            """
        )
        
        ## Update memory
        self.history.append(
            ChatMessage(
                role = MessageRole.ASSISTANT,
                content = "FINAL ANSWER: " + str(response)
            )
        )
        
        return StopEvent(result = str(response))

In [8]:
draw_all_possible_flows(MixtureOfAnswers, filename="workflow.html")

workflow.html
